# 1. Prepare Env

## 1.1. Installing Required Packages.

In [1]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools
!pip install --upgrade tf-slim
!conda install pandas
!conda install -c menpo opencv

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Requirement already up-to-date: tf-slim in /home/vsmart/anaconda3/envs/tf1gpu/lib/python3.7/site-packages (1.1.0)
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## 1.2. Importing Libraries

In [1]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import sys,os,os.path
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

%env MODELS_PATH=/media/vsmart/DATA/dannv5/DataTrain/
%env RACCOON_DATA_PATH=/media/vsmart/DATA/dannv5/DataTrain/raccoon

env: MODELS_PATH=/media/vsmart/DATA/dannv5/DataTrain/
env: RACCOON_DATA_PATH=/media/vsmart/DATA/dannv5/DataTrain/raccoon


## 1.3. Downloading Tensorflow model

In [ ]:
MODELS_DIR = os.environ.get('MODELS_PATH')
%cd $MODELS_DIR
# downloads tensorflow models 
!git clone --q https://github.com/tensorflow/models.git

## 1.4. Setup PYTHONPATH

In [2]:
# goto /models/research
MODELS_DIR = os.environ.get('MODELS_PATH')
%cd $MODELS_DIR/models/research
# compils the proto buffers
!protoc object_detection/protos/*.proto --python_out=.
# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
os.environ['PYTHONPATH']+=':./:./slim/'
!printenv PYTHONPATH

/media/vsmart/DATA/dannv5/DataTrain/models/research
object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.
/media/vsmart/DATA/dannv5/DataTrain/models/research/:/media/vsmart/DATA/dannv5/DataTrain/models/research/slim/:./:./slim/


In [3]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

# 2. Data preprocessing

## 2.1. Splitting the images into training & testing

Note: The images inside images don’t need to be split, only the .xml files.

In [ ]:
DATA_DIR = os.environ.get('MODELS_PATH')
%cd $DATA_DIR

In [ ]:
#creating two dir for training and testing
!mkdir test_labels train_labels

In [ ]:
# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 400/2000 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -40 | xargs -I{} mv {} test_labels/

# Moves the rest of labels '1600' labels to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

## 2.2. Create CSV

In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

#Generating Tf record

*   Generating two TFRecords files for the training and testing CSVs.
*   Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically





In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util


#change this to the base directory where your data/ is 
data_base_url = '/gdrive/My Drive/raccoon/data/'

#location of images
image_dir = 'images/'

def class_text_to_int(row_label):
  if row_label == 'raccoon':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
	  encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size
	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(row['xmin'] / width)
		xmaxs.append(row['xmax'] / width)
		ymins.append(row['ymin'] / height)
		ymaxs.append(row['ymax'] / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(class_text_to_int(row['class']))

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
	return tf_example


In [ ]:
# %cd /gdrive/'My Drive'/Data/raccoon/data
#creates tfrecord for both csv's

for csv in ['train_labels', 'test_labels']:
  print(data_base_url)
  writer = tf.io.TFRecordWriter(csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), csv + '.record')
  print('Successfully created the TFRecords: {}'.format(csv + '.record'))

## 2.4. Selecting and Downloading a Pre-trained Model

In [ ]:
MODELS_DIR = os.environ.get('MODELS_PATH')
%cd $MODELS_DIR/models/research

In [ ]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'

In [ ]:
#the distination folder where the model will be saved
#change this if you have a different working dir
MODELS_DIR = os.environ.get('MODELS_PATH')
DEST_DIR = $MODELS_DIR/models/research/pretrained_model

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

# 3. Train steps

In [4]:
MODELS_DIR = os.environ.get('MODELS_PATH')
%cd $MODELS_DIR/models/research

/media/vsmart/DATA/dannv5/DataTrain/models/research


## 3.1. Configuring the Training Pipeline



```
Edit config file 
*   model {} > ssd {}: change num_classes to the number of classes you have.
*   train_config {}: change fine_tune_checkpoint to the checkpoint file path.
*   train_input_reader {}: set the path to the train_labels.record and the label map pbtxt file.
*   eval_input_reader {}: set the path to the test_labels.record and the label map pbtxt file
```

In [5]:
#path to the config file
!cat object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

## 3.2. Tensorboard

To use Tensorboard on Colab, we need to use it through ngrok. Get it by running

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

specify where the log files are stored and we configure a link to view Tensorboard

In [ ]:
#the logs that are created while training 
LOG_DIR = "training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## 3.3. Training

In [10]:
MODELS_DIR = os.environ.get('MODELS_PATH')
%cd $MODELS_DIR/models

/media/vsmart/DATA/dannv5/DataTrain/models


In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
!python3 research/object_detection/legacy/train.py --logtostderr \
    --pipeline_config_path=$MODELS_DIR/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --train_dir=training/


2020-06-26 14:57:18.723180: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-06-26 14:57:18.746918: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3199980000 Hz
2020-06-26 14:57:18.747244: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55674b252120 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-26 14:57:18.747256: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-26 14:57:18.747839: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-26 14:57:18.765520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-26

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
W0626 14:57:20.364418 140597991786304 deprecation.py:323] From /media/vsmart/DATA/dannv5/DataTrain/models/research/object_detection/builders/dataset_builder.py:48: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0626 14:57:20.398318 140597991786304 deprecation.py:323] From /media/vsmart/DATA/dann

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W0626 14:57:28.803151 140597991786304 deprecation.py:323] From /home/vsmart/anaconda3/envs/tf1gpu/lib/python3.7/site-packages/tf_slim/learning.py:734: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2020-06-26 14:57:29.531158: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-26 14:57:29.531390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1650 major: 7 minor: 5 memoryClockRate(GHz): 1.725
pciBusID: 0000:01:00.0
2020-06-26 14:57:29.531441: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10

INFO:tensorflow:global step 16: loss = 7.3026 (0.104 sec/step)
I0626 14:57:46.132500 140597991786304 learning.py:512] global step 16: loss = 7.3026 (0.104 sec/step)
INFO:tensorflow:global step 17: loss = 8.0858 (0.102 sec/step)
I0626 14:57:46.235371 140597991786304 learning.py:512] global step 17: loss = 8.0858 (0.102 sec/step)
INFO:tensorflow:global step 18: loss = 9.1788 (0.100 sec/step)
I0626 14:57:46.336397 140597991786304 learning.py:512] global step 18: loss = 9.1788 (0.100 sec/step)
INFO:tensorflow:global step 19: loss = 8.2934 (0.105 sec/step)
I0626 14:57:46.442257 140597991786304 learning.py:512] global step 19: loss = 8.2934 (0.105 sec/step)
INFO:tensorflow:global step 20: loss = 7.5602 (0.116 sec/step)
I0626 14:57:46.559473 140597991786304 learning.py:512] global step 20: loss = 7.5602 (0.116 sec/step)
INFO:tensorflow:global step 21: loss = 8.5797 (0.107 sec/step)
I0626 14:57:46.667597 140597991786304 learning.py:512] global step 21: loss = 8.5797 (0.107 sec/step)
INFO:tenso

INFO:tensorflow:global step 66: loss = 6.5271 (0.103 sec/step)
I0626 14:57:51.585451 140597991786304 learning.py:512] global step 66: loss = 6.5271 (0.103 sec/step)
INFO:tensorflow:global step 67: loss = 5.7764 (0.100 sec/step)
I0626 14:57:51.686231 140597991786304 learning.py:512] global step 67: loss = 5.7764 (0.100 sec/step)
INFO:tensorflow:global step 68: loss = 5.3230 (0.099 sec/step)
I0626 14:57:51.785961 140597991786304 learning.py:512] global step 68: loss = 5.3230 (0.099 sec/step)
INFO:tensorflow:global step 69: loss = 8.2343 (0.106 sec/step)
I0626 14:57:51.892826 140597991786304 learning.py:512] global step 69: loss = 8.2343 (0.106 sec/step)
INFO:tensorflow:global step 70: loss = 6.5616 (0.102 sec/step)
I0626 14:57:51.995482 140597991786304 learning.py:512] global step 70: loss = 6.5616 (0.102 sec/step)
INFO:tensorflow:global step 71: loss = 8.5077 (0.100 sec/step)
I0626 14:57:52.096561 140597991786304 learning.py:512] global step 71: loss = 8.5077 (0.100 sec/step)
INFO:tenso

...

INFO:tensorflow:global step 1895: loss = 2.8162 (0.104 sec/step)
I0626 15:01:00.490451 140597991786304 learning.py:512] global step 1895: loss = 2.8162 (0.104 sec/step)
INFO:tensorflow:global step 1896: loss = 2.6174 (0.103 sec/step)
I0626 15:01:00.594049 140597991786304 learning.py:512] global step 1896: loss = 2.6174 (0.103 sec/step)
INFO:tensorflow:global step 1897: loss = 3.3231 (0.100 sec/step)
I0626 15:01:00.695609 140597991786304 learning.py:512] global step 1897: loss = 3.3231 (0.100 sec/step)
INFO:tensorflow:global step 1898: loss = 4.2783 (0.101 sec/step)
I0626 15:01:00.797148 140597991786304 learning.py:512] global step 1898: loss = 4.2783 (0.101 sec/step)
INFO:tensorflow:global step 1899: loss = 3.5052 (0.103 sec/step)
I0626 15:01:00.900905 140597991786304 learning.py:512] global step 1899: loss = 3.5052 (0.103 sec/step)
INFO:tensorflow:global step 1900: loss = 2.0335 (0.100 sec/step)
I0626 15:01:01.001538 140597991786304 learning.py:512] global step 1900: loss = 2.0335 (0.

INFO:tensorflow:global step 1944: loss = 3.0899 (0.103 sec/step)
I0626 15:01:05.567479 140597991786304 learning.py:512] global step 1944: loss = 3.0899 (0.103 sec/step)
INFO:tensorflow:global step 1945: loss = 3.0069 (0.101 sec/step)
I0626 15:01:05.668965 140597991786304 learning.py:512] global step 1945: loss = 3.0069 (0.101 sec/step)
INFO:tensorflow:global step 1946: loss = 2.8839 (0.098 sec/step)
I0626 15:01:05.768401 140597991786304 learning.py:512] global step 1946: loss = 2.8839 (0.098 sec/step)
INFO:tensorflow:global step 1947: loss = 3.9118 (0.102 sec/step)
I0626 15:01:05.871469 140597991786304 learning.py:512] global step 1947: loss = 3.9118 (0.102 sec/step)
INFO:tensorflow:global step 1948: loss = 2.2897 (0.100 sec/step)
I0626 15:01:05.972146 140597991786304 learning.py:512] global step 1948: loss = 2.2897 (0.100 sec/step)
INFO:tensorflow:global step 1949: loss = 3.0764 (0.103 sec/step)
I0626 15:01:06.076569 140597991786304 learning.py:512] global step 1949: loss = 3.0764 (0.

INFO:tensorflow:global step 1993: loss = 6.9252 (0.104 sec/step)
I0626 15:01:10.747834 140597991786304 learning.py:512] global step 1993: loss = 6.9252 (0.104 sec/step)
INFO:tensorflow:global step 1994: loss = 2.1382 (0.097 sec/step)
I0626 15:01:10.845428 140597991786304 learning.py:512] global step 1994: loss = 2.1382 (0.097 sec/step)
INFO:tensorflow:global step 1995: loss = 1.9257 (0.105 sec/step)
I0626 15:01:10.951728 140597991786304 learning.py:512] global step 1995: loss = 1.9257 (0.105 sec/step)
INFO:tensorflow:global step 1996: loss = 2.8221 (0.100 sec/step)
I0626 15:01:11.052503 140597991786304 learning.py:512] global step 1996: loss = 2.8221 (0.100 sec/step)
INFO:tensorflow:global step 1997: loss = 2.7718 (0.101 sec/step)
I0626 15:01:11.154421 140597991786304 learning.py:512] global step 1997: loss = 2.7718 (0.101 sec/step)
INFO:tensorflow:global step 1998: loss = 3.1792 (0.101 sec/step)
I0626 15:01:11.256473 140597991786304 learning.py:512] global step 1998: loss = 3.1792 (0.

INFO:tensorflow:global step 2042: loss = 4.8501 (0.105 sec/step)
I0626 15:01:15.896078 140597991786304 learning.py:512] global step 2042: loss = 4.8501 (0.105 sec/step)
INFO:tensorflow:global step 2043: loss = 2.4305 (0.100 sec/step)
I0626 15:01:15.997230 140597991786304 learning.py:512] global step 2043: loss = 2.4305 (0.100 sec/step)
INFO:tensorflow:global step 2044: loss = 2.1489 (0.105 sec/step)
I0626 15:01:16.103374 140597991786304 learning.py:512] global step 2044: loss = 2.1489 (0.105 sec/step)
INFO:tensorflow:global step 2045: loss = 2.5527 (0.099 sec/step)
I0626 15:01:16.203791 140597991786304 learning.py:512] global step 2045: loss = 2.5527 (0.099 sec/step)
INFO:tensorflow:global step 2046: loss = 2.2404 (0.101 sec/step)
I0626 15:01:16.306123 140597991786304 learning.py:512] global step 2046: loss = 2.2404 (0.101 sec/step)
INFO:tensorflow:global step 2047: loss = 2.2500 (0.100 sec/step)
I0626 15:01:16.407330 140597991786304 learning.py:512] global step 2047: loss = 2.2500 (0.

INFO:tensorflow:global step 2091: loss = 2.4063 (0.105 sec/step)
I0626 15:01:20.914281 140597991786304 learning.py:512] global step 2091: loss = 2.4063 (0.105 sec/step)
INFO:tensorflow:global step 2092: loss = 4.0297 (0.100 sec/step)
I0626 15:01:21.014882 140597991786304 learning.py:512] global step 2092: loss = 4.0297 (0.100 sec/step)
INFO:tensorflow:global step 2093: loss = 2.1524 (0.101 sec/step)
I0626 15:01:21.116703 140597991786304 learning.py:512] global step 2093: loss = 2.1524 (0.101 sec/step)
INFO:tensorflow:global step 2094: loss = 2.4675 (0.101 sec/step)
I0626 15:01:21.218778 140597991786304 learning.py:512] global step 2094: loss = 2.4675 (0.101 sec/step)
INFO:tensorflow:global step 2095: loss = 2.9804 (0.108 sec/step)
I0626 15:01:21.328028 140597991786304 learning.py:512] global step 2095: loss = 2.9804 (0.108 sec/step)
INFO:tensorflow:global step 2096: loss = 2.7845 (0.101 sec/step)
I0626 15:01:21.429839 140597991786304 learning.py:512] global step 2096: loss = 2.7845 (0.

# 4. Export Model

# 5. Inference 